## 第2週 4日目

ディープ・リサーチ：古典的なクロス・ビジネス・エージェントのユースケースの1つ！これは巨大です。

### 概要
**最新のAIエージェント・フレームワーク（2025年版）**に関する**ディープ・リサーチ**を自動化するエージェントを構築・実行する。

- 準備：ライブラリのインポート、環境変数のロード
- WebSearchTool を使う search_agent の作成
- 構造化出力を行う planner_agent の作成
- メール送信ツールを使う email_agentの作成
- 構造化出力を行う writer_agent の作成
- 上記のエージェントを実行する関数群を定義
- 関数を順次呼び出しエージェント・フローを実行

ログを見ると以下のように動作したことが確認できる。

- Starting research...
- Planning searches...
  - message = "Latest AI Agent frameworks in 2025"
  - Will perform 3 searches
  - Finished Planning
- Searching...
  - Searching "latest AI agent frameworks 2025"...
  - Searching "top AI agent tools 2025"...
  - Searching "AI agent frameworks trends 2025"...
  - Finished searching
- Thinking about report...
  Finished writing report
- Writing email...
  Email sent
- Hooray!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">商業的影響</h2>
            <span style="color:#00bfff;">
                Deep Researchエージェントは、あらゆるビジネス分野や日々の業務に幅広く応用できます。ご自身でも活用いただけます！
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
import asyncio
from typing import Dict
from pydantic import BaseModel, Field

from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings

import resend # import sendgrid
#from sendgrid.helpers.mail import Mail, Email, To, Content
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAIホストツール

OpenAI Agents SDKには、次のホストツールが含まれています。

`WebSearchTool`  を使用すると、エージェントがWebを検索できます。  
`FileSearchTool` を使用すると、OpenAIベクターストアから情報を取得できます。  
`ComputerTool`   は、スクリーンショットの撮影やクリックなどのタスクを自動化することができます。

### 重要なメモ - WebSearchTool の API チャージ

これには、OpenAI WebSearchTool の呼び出しごとに2.5セントの費用がかかります。次の2つのラボでは、最大2-3ドルになります。他のプラットフォームで無料および低コストの検索ツールを使用するので、コストが懸念される場合は、これを実行して自由にスキップしてください。また、学生の Christian W. は、OpenAIが1回のコールの複数の検索に対して請求する場合があるため、通話ごとに2.5セント以上の費用がかかる場合があると指摘しました。

コストはこちら: https://platform.openai.com/docs/pricing#web-search

## WebSearchTool を使う search_agent

In [3]:
# 指示「あなたは研究助手です。与えられた検索語でウェブを検索し、その結果を簡潔に要約してください。
# 要約は2～3段落、300語未満でなければなりません。要点を捉えてください。簡潔に記述してください。完全な文や正確な文法は必要ありません。
# これはレポートを作成する人が参照するものなので、要点を捉え、余分な部分は省くことが重要です。要約以外のコメントは含めないでください。」
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],     # sizeはWebからの取得コンテキスト量。
    model="gpt-4o-mini",                                  # 費用を抑えたいので -mini を指定。
    model_settings=ModelSettings(tool_choice="required"), # 任意のツールの使用を必須に指定
)

In [4]:
# 2025年の最新AIエージェントフレームワーク
message = "Latest AI Agent frameworks in 2025"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Search"):
    # WebSearchToolを使用して「2025年の最新AIエージェントフレームワーク」について調査
    result = await Runner.run(search_agent, message)

# 結果をMarkdownで表示
display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**Amazon Bedrock AgentCore**: Launched by AWS, AgentCore provides a suite of tools for enterprise-scale AI agent development, including scalable serverless deployment, context management, and secure service access. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

**Microsoft AutoGen**: This open-source framework facilitates the creation of multi-agent systems, enabling seamless communication and collaboration among agents to solve complex problems. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))

**Eliza**: A Web3-friendly AI agent operating system that integrates blockchain data and smart contracts, allowing for the deployment of decentralized applications. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**Agent Lightning**: A flexible framework that enables reinforcement learning-based training of large language models for any AI agent, supporting complex interaction logic and dynamic workflows. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**AutoAgent**: A fully automated, zero-code framework that allows users to create and deploy LLM agents through natural language, making AI agent development accessible without programming skills. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

**Symphony**: A decentralized multi-agent system that enables lightweight LLMs on consumer-grade GPUs to coordinate, offering a scalable and fault-tolerant orchestration with low overhead. ([arxiv.org](https://arxiv.org/abs/2508.20019?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent technology, catering to diverse needs from enterprise solutions to decentralized applications. 

### いつものように、トレースを見てください

https://platform.openai.com/traces

## 構造化出力を行う planner_agent
ウェブ検索（理由、クエリ）のリストを構造化出力するエージェント

In [5]:
# WebSearchToolのコストに関する上記のメモを参照してください

HOW_MANY_SEARCHES = 3

# あなたは役に立つ研究アシスタントです。
# クエリが与えられた場合、クエリに最適な回答を得るために実行するウェブ検索のセットを考え出してください。
# 検索対象となる {HOW_MANY_SEARCHES} 個の用語を出力してください。
INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Pydanticを使用して、私たちの応答のスキーマを定義します - これは「構造化された出力」として知られています
# 学生のWes C.に大いに感謝して、これで厄介なバグを発見して修正してくれました！

# ウェブ検索（理由、クエリ）
class WebSearchItem(BaseModel):
    # この検索がクエリにとって重要である理由。
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    # Web 検索に使用する検索語句。
    query: str = Field(description="The search term to use for the web search.")

# ※ 余談だが、reasonを入れておくことで、CoT的に、queryのクオリティが高まるとのこと。

# ウェブ検索（理由、クエリ）のリスト
class WebSearchPlan(BaseModel):
    # クエリに最適な回答を得るために実行する Web 検索のリスト。
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

# ウェブ検索（理由、クエリ）のリストを構造化出力するエージェント
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
# 2025年の最新AIエージェントフレームワーク
message = "Latest AI Agent frameworks in 2025"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Search"):
    # ウェブ検索のセット（理由、クエリ）を構造化出力
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find comprehensive overviews or articles on the latest AI agent frameworks released or announced in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To discover specific frameworks that have gained popularity or are recommended in the AI community in 2025.', query='top AI frameworks 2025'), WebSearchItem(reason='To identify reviews or comparisons of different AI agent frameworks available in 2025.', query='comparison of AI agent frameworks 2025')]


### いつものように、トレースを見てください

https://platform.openai.com/traces

## メール送信ツールを使う email_agent

In [7]:
# メール送信ツール
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    
    #sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    resend.api_key=os.getenv("RESEND_API_KEY")
    
    #from_email = Email("ed@edwarddonner.com")  # 検証済みの送信者に変更します
    #to_email = To("ed.donner@gmail.com")  # 受信者に変更します
    #content = Content("text/html", html_body)
    #mail = Mail(from_email, to_email, subject, content).get()
    #sg.client.mail.send.post(request_body=mail)
    
    response = resend.Emails.send({
        "from": "onboarding@resend.dev",
        "to": "nishi_74322014@ksj.biglobe.ne.jp",
        "subject": subject,
        "html": html_body
    })

    return {"status": "success"}

send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f6268abfe20>, strict_json_schema=True, is_enabled=True)

In [8]:
# 詳細なレポートに基づいて、見やすくフォーマットされたHTMLメールを送信できます。
# 詳細なレポートが提供されます。ツールを使用して、レポートを適切な件名を付けた、見やすく見やすいHTMLに変換したメールを1通送信してください。
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

# メール送信エージェント
email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

## 構造化出力を行う writer_agent
研究報告書（要約、本文、研究提案）を構造化出力するエージェント

In [9]:
# あなたは、ある研究テーマについて、まとまりのある報告書を書くという任務を負った上級研究者です。
# オリジナルの研究テーマと、研究アシスタントが行った初期調査が提供されます。
# まず、報告書の構成と流れを説明する報告書のアウトラインを作成してください。
# 次に、報告書を作成し、最終成果物として提出してください。
# 最終成果物はマークダウン形式で、長く詳細な内容にしてください。
# 5～10ページ、少なくとも1000語の内容を目指してください。

INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

# 報告情報
class ReportData(BaseModel):
    # 調査結果を 2 ～ 3 文で簡潔に要約
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    # 最終報告書
    markdown_report: str = Field(description="The final report")
    # さらに研究するための提案トピック
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

# 報告書作成エージェント
writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## エージェント関数

### 次の3つの関数は、planner_agent と search_agent を使用して、検索を計画して実行します

In [10]:
# planner_agentを実行する。
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    # planner_agent を使用して、クエリに対して実行する検索を計画。
    
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    print("Finished Planning")
    
    return result.final_output

# search_planを並列実行する。
async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    # 検索プラン内の各項目に対してsearch()を呼び出す。
    
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    
    return results

# search_planの一部を単体実行する。
async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    # search_agent を使用して、検索プラン内の各項目のウェブ検索を実行。

    print(f"Searching {item.query}...") # 少々ログを強化した。
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    
    return result.final_output

### 次の2つの関数は、writer_agent と email_agent を使用して、レポートを書き電子メールで送信します

In [11]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results """
    # writer_agent を使用して、検索結果に基づいてレポートを作成する
    
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    # email_agent を使用してレポートを添付したメールを送信する
    
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    
    return report

### ショータイム！
わりと大掛かりな、エージェント・フロー

In [12]:
# 2025年の最新AIエージェントフレームワーク
query ="Latest AI Agent frameworks in 2025"

# OpenAI API PF → Dashboard → Logs → Traces の 
with trace("Research trace"):
    
    # わりと大掛かりな、エージェント・フロー
    print("Starting research...")

    # エージェントでプランを作成
    search_plan = await plan_searches(query)
    
    # プランをエージェントで並列実行
    search_results = await perform_searches(search_plan)

    # 結果を受けてレポートを作成
    report = await write_report(query, search_results)

    # エージェントでメールを送信
    await send_email(report)  
    
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Finished Planning
Searching...
Searching latest AI agent frameworks 2025...
Searching top AI agent tools 2025...
Searching AI agent frameworks trends 2025...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### いつものように、トレースを見てください

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">進捗おめでとうございます！そしてお願いがあります。</h2>
            <span style="color:#00cc00;">コースの重要な段階に到達しました。最新のエージェントフレームワークの一つを使って、価値あるエージェントを作成できました。スキルアップを果たし、新たなビジネスチャンスを切り開きました。この成功をお祝いする時間を取りましょう！<br/><br/>お願いがあります。もしこれを言わないと編集者に叩かれてしまうでしょう。Udemyでコースを評価していただけると大変助かります。評価はUdemyがこのコースを他のユーザーに公開するかどうかを決める最も重要な要素であり、大きな違いを生みます。<br/><br/>もう一つお知らせです。ご希望であれば、<a href="https://www.linkedin.com/in/eddonner/">LinkedInで私と繋がってください！</a> コースの進捗状況を投稿したい場合は、私をタグ付けしてください。露出を高めるために私が協力させていただきます。</span>
        </td>
    </tr>